# Variational Quantum Eigensolver for 1-qubit system  

This is an attempt at finding the lowest eigenvalue for a 2 x 2 matrix using VQE

In [1]:
from qiskit import *
import numpy as np
from scipy import optimize

In [2]:
toy_hamiltonian = np.array([[3,  1], [1, -1]], dtype=complex)

print(toy_hamiltonian)

pauliI = np.array([[1,  0], [0,  1]], dtype=complex)
pauliX = np.array([[0,  1], [1,  0]], dtype=complex)
pauliY = np.array([[0, -1j], [1j,  0]], dtype=complex)
pauliZ = np.array([[1,  0], [0, -1]], dtype=complex) 
pauliMatrices = [["I", pauliI], ["X", pauliX], ["Y", pauliY],   ["Z", pauliZ]]

coefficientDict = {}
nQubits = 0

for pauliMatrix in pauliMatrices:
    coefficientDict[f"{pauliMatrix[0]}"] = 0.5 * np.trace(np.matmul(pauliMatrix[1], toy_hamiltonian))
    if abs(coefficientDict[f"{pauliMatrix[0]}"]) > 0 and pauliMatrix[0] != 'I':
        nQubits += 1

[[ 3.+0.j  1.+0.j]
 [ 1.+0.j -1.+0.j]]


In [3]:
def prepareCircuit(nQubits, coefficientDict, parameters):
    theta1, theta2 = parameters
    circuit = QuantumCircuit(nQubits, nQubits)

    for i in range(nQubits):
        # Adding the ansatz to the circuit
        circuit.ry(theta1, i)
        circuit.rx(theta2, i)

    qubitNumber = 0

    if abs(coefficientDict['X']) > 0:
        circuit.ry(-np.pi / 2, qubitNumber)
        qubitNumber += 1

    if abs(coefficientDict['Y']) > 0:
        circuit.rx(np.pi / 2, qubitNumber)
        qubitNumber += 1

    circuit.barrier()

    for i in range(nQubits):
        circuit.measure(i, i)

    circuit.draw(output='mpl')
    return circuit

In [4]:
def vqe(parameters):
    # Preparing the circuit
    circuit = prepareCircuit(nQubits, coefficientDict, parameters)

    # Performing the measurement to get the costs
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=8192)
    result = job.result()
    counts = result.get_counts()

    # Calculating the energy
    energy = 0
    index = 0

    for coefficient in coefficientDict:
        if abs(coefficientDict[coefficient]) > 0:
            if coefficient == 'I':
                energy += coefficientDict[coefficient]
            else:
                for count in counts:
                    count = count[::-1]
                    if count[index] == '0':
                        energy += coefficientDict[coefficient] * (counts[count[::-1]] / 8192)
                    else:
                        energy -= coefficientDict[coefficient] * (counts[count[::-1]] / 8192)
                index += 1
    return np.real(energy)

In [5]:
theta1 = (np.random.rand(1) - 0.5) * 2 * np.pi
theta2 = (np.random.rand(1) - 0.5) * 2 * np.pi
parameters = np.array([theta1, theta2])
tol = 1e-5

exacteigenvalues = np.linalg.eigvals(toy_hamiltonian)
min_eigenvalue = np.min(exacteigenvalues)

print(f'Exact eigenvalue is {np.real(min_eigenvalue)}')

result = optimize.minimize(vqe, parameters, method='Powell', tol=tol)

print(f"Eigenvalue obtained by using VQE algorithm with classical optimisation using Powell's method is", result.fun)

Exact eigenvalue is -1.2360679774997896
Eigenvalue obtained by using VQE algorithm with classical optimisation using Powell's method is -1.249267578125
